In [12]:
import pickle
from IPython.display import display, HTML
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import datetime as dt

In [13]:
with open('Formatuota_lentele.pkl', 'rb') as file:
    df_Elektros_lentele = pickle.load(file)

In [14]:
def duomenu_paruosimas(df):
    df = df.reset_index()
    df['Data'] = pd.to_datetime(df['Data'], format='%Y-%m-%d')
    df['Data'] = df['Data'].dt.strftime('%Y-%m')
    df = df.set_index('Data')
    return df

def duomenu_pridejimas(df, df_naujas, stulp_pav):
    df_naujas = df_naujas.rename(columns={'Value': stulp_pav})
    df_sudetas = df.merge(df_naujas, on='Data', how='outer')
    df_sudetas = df_sudetas.sort_values(by='Data')
    df_sudetas = df_sudetas.infer_objects(copy=True).ffill().bfill()
    return df_sudetas

def nuskaitymas_is_excel(failas):
    df = pd.read_excel(failas, engine='openpyxl')
    df['Date'] = pd.to_datetime(df['Date'], format='%y - %b')
    df['Date'] = df['Date'].dt.strftime('%Y-%m')
    df.set_index('Date', inplace=True)
    df.index.name = 'Data'
    return df

***1. Pildymas - Elektros kaina Biržoje***  
Rankiniu būdu suvedamos NORD POOL kainos iš saito. Selenium nenaudojau.  
"https://www.nordpoolgroup.com/en/market-data12/Dayahead/Area-Prices/LT/Monthly/?view=table"  
Arba atsidarome iš excel failo mėnesines vertes

In [15]:
# k_2015 = 41.92 # 2015 EUR/mwh
# k_2016 = 36.54 # 2016 EUR/mwh
# k_2017 = 35.13 # 2017 EUR/mwh
# k_2018 = 50.00 # 2018 EUR/mwh
# k_2019 = 46.12 # 2019 EUR/mwh
# k_2020 = 34.04 # 2020 EUR/mwh
# k_2021 = 90.45 # 2021 EUR/mwh
# k_2022 = 230.23 # 2022 EUR/mwh
# k_2023 = 94.44 # 2023 EUR/mwh

# k_2024_6 = 91.64 # 2024 Birželio EUR/mwh	
# k_2024_5 = 75.85	
# k_2024_4 = 60.25	
# k_2024_3 = 68.21	
# k_2024_2 = 74.78	
# k_2024_1 = 117.41 # 2024 Sausio EUR/mwh

# k_2024_list = [k_2024_1, k_2024_2, k_2024_3, k_2024_4, k_2024_5, k_2024_6]
# k_2024 = round(np.mean(k_2024_list), 2)  # Vedu 6 mėn vidurkį
# Nord_pool_list = [k_2015, k_2016, k_2017, k_2018, k_2019, k_2020, k_2021, k_2022, k_2023, k_2024]



In [16]:
Elektros_kainos_df = nuskaitymas_is_excel('Elektros_kainos_men.xlsx') # Failą sukūriau rankiniu būdu
Elektros_kainos_df['Value'] = Elektros_kainos_df['Value'] / 1000
Elektros_kainos_df

,Value
Data,
2024-06,0.09164
2024-05,0.07585
2024-04,0.06025
2024-03,0.06821
2024-02,0.07478
...,...
2015-05,0.03736
2015-04,0.03561
2015-03,0.03222


***2. Pildymas - Infliacijos rodiklis***  
Metinė infliacija  
Arba atsidarome iš excel failo mėnesines vertes

In [17]:
# in_2015 = -0.68
# in_2016 = 0.68
# in_2017 = 3.72
# in_2018 = 2.52
# in_2019 = 2.24
# in_2020 = 1.06
# in_2021 = 4.62
# in_2022 = 18.86
# in_2023 = 8.60
# in_2024 = 1.46 # Prognozuojamas
# Infliacija_LTU_list = [in_2015, in_2016, in_2017, in_2018, in_2019, in_2020, in_2021, in_2022, in_2023, in_2024]

In [18]:
Infliacija_df = nuskaitymas_is_excel('Infliacija_menesiu.xlsx') # Failą sukūriau rankiniu būdu
Infliacija_df

,Value
Data,
2024-06,0.1
2024-05,0.3
2024-04,0.1
2024-03,-0.2
2024-02,0.7
...,...
2015-05,0.5
2015-04,0.7
2015-03,0.8


***3. Pildymas - Vidutinė metinė temperatūra***  

In [19]:
# %pip install xlrd

df_temperatura = pd.read_excel('Temp_Krituliai_1991_2020_kasdieniai.xls', sheet_name='Temp_vidutinė', engine='xlrd') # Parsiųsta iš xls failo
df_2015_2020 = df_temperatura[df_temperatura['TVID'] >= 2015].copy()
df_2015_2020['Value'] = df_2015_2020.iloc[:, 3:].mean(axis=1)
monthly_means = df_2015_2020.groupby(['TVID', 'mėnuo'])['Value'].mean()
monthly_means = monthly_means.round(2).reset_index()
monthly_means['Data'] = monthly_means['TVID'].astype(str) + '-' + monthly_means['mėnuo'].apply(lambda x: f'{x:02d}')
monthly_means = monthly_means.drop(columns=['TVID', 'mėnuo'])
df_2015_2020 = monthly_means.set_index('Data')

df_2021_2024 = nuskaitymas_is_excel('Temp_menesiu_2021_2024.xlsx') # Failą sukūriau rankiniu būdu
Temperatura_df = pd.concat([df_2015_2020, df_2021_2024])
Temperatura_df

,Value
Data,
2015-01,-0.38
2015-02,0.09
2015-03,4.16
2015-04,6.67
2015-05,11.06
...,...
2021-05,11.10
2021-04,5.90
2021-03,1.60


***4. Pildymas - Elektros gamyba kas mėnesį***  
Parsisiunčiu 'data' failą iš čia:  
https://www.litgrid.eu/index.php/sistema/elektros-energetikos-sistemos-informacija/elektros-gamybos-ir-vartojimo-balanso-duomenys/2287

In [20]:
# Funkcija kuri padalins ketvirtį į tris mėnesius, kurių procentinė dalis bus 'random' 30-36 procentų rėžiuose.
def Procento_generatorius():
    while True:
        p1 = np.random.uniform(30, 36)
        p2 = np.random.uniform(30, 36)
        p3 = 100 - p1 - p2  # Trijų mėnesių procentinė suma privalo būti 100%
        if 30 <= p3 <= 36:  # Bet trečio mėnesio likučio procentinė išraiška turi taip pat tilpti į 30-36 procento rėžį
            return [p1, p2, p3]

# Lentelės įsikėlimas suformuoti dataframe ir palikti reikalingas vertes.
df_paruosimas = pd.read_excel(r'Databases\Ketvirtiniai.xlsx', dtype='unicode')
df_paruosimas = df_paruosimas.iloc[:2, :] # Trumpinu dataframe iki man reikalingų eilučių
df_paruosimas = df_paruosimas.drop(df_paruosimas.columns[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]], axis=1) # Filtruoju reikalingą rėžį 
df_paruosimas = df_paruosimas.drop([0])

df_paruosimas.rename(columns={df_paruosimas.columns[0]: 'Pavadinimas'}, inplace=True)
for col in df_paruosimas.columns[1:]:
    df_paruosimas[col] = pd.to_numeric(df_paruosimas[col], errors='coerce').round(2)

# Lentelės 'apvertimas' taip kad metų ketvirčiai būtų eilutėse, o stulpelis atitiktų vertę
df_apverstas = df_paruosimas.melt(id_vars=['Pavadinimas'], var_name='Data', value_name='Value')
df_apverstas = df_apverstas[~df_apverstas['Data'].str.contains(r'\d{3}', na=False)].reset_index()

# Naudoju universalius pavadinimus, kad lengvai pridėtų į Dataframe
Pagaminta_elektra_df = pd.DataFrame(columns=['Data', 'Value'])

for i, eilute in df_apverstas.iterrows():
    ketvirtis = eilute['Data']
    verte = eilute['Value']
    ketv_procentai = Procento_generatorius() # Sukuriamas random trijų elementų list'as
    # Ketvirčio ciklas mėnesiams (3 iteracijos per ketvirtį)
    for j, procentas in enumerate(ketv_procentai):
        if i % 4 == 0:
            m = i
        j += (i * 3) + 1   
        metai = df_apverstas.iloc[m, 2]
        menuo = ((j - 1) % 12 + 1) 

        date_str = f"{metai}-{menuo}"
        date = pd.to_datetime(date_str, format='%Y-%m')
        formatuota_data = date.strftime('%Y-%m')

        Pagaminta_elektra_df.loc[j, 'Data'] = formatuota_data
        Pagaminta_elektra_df.loc[j, 'Value'] = round(verte * (procentas / 100), 3)

# Pridedu trūkstamas 6 eilutes
menesiai = ['2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06']
vertes = [0.699, 0.564, 0.611, 0.633, 0.639, 0.592]
df_2024 = pd.DataFrame({'Data': menesiai, 'Value': vertes})

Pagaminta_elektra_df = pd.concat([Pagaminta_elektra_df, df_2024])

Pagaminta_elektra_df.set_index('Data', inplace=True)
display(HTML(Pagaminta_elektra_df.to_html())) # Padaro scrollable df



,Value
Data,
2015-01,0.359
2015-02,0.359
2015-03,0.361
2015-04,0.41
2015-05,0.348
2015-06,0.391
2015-07,0.375
2015-08,0.374
2015-09,0.411


***Rezultatų tikrinimas***

In [21]:
df_darbinis = duomenu_paruosimas(df_Elektros_lentele)
df_darbinis = duomenu_pridejimas(df_darbinis, Elektros_kainos_df, 'Nord pool kaina')
df_darbinis = duomenu_pridejimas(df_darbinis, Infliacija_df, 'Metinė infliacija')
df_darbinis = duomenu_pridejimas(df_darbinis, Temperatura_df, 'Vidutinė Temperatūra C')
df_darbinis = duomenu_pridejimas(df_darbinis, Pagaminta_elektra_df, 'Pagaminta elektra GWh')

df_darbinis
columns = ['Dieninė dedamoji', 'Naktinė dedamoji', 'Pastovioji dedamoji. EUR/mėn.',
       'Viena laiko zona', 'Nord pool kaina', 'Metinė infliacija',
       'Vidutinė Temperatūra C', 'Pagaminta elektra GWh']
df_darbinis[columns] = df_darbinis[columns].apply(pd.to_numeric, errors='coerce')


In [22]:
with open('Paruosti_duomenys.pkl', 'wb') as file:
    pickle.dump(df_darbinis, file)